## Importa bibliotecas

In [5]:
import pandas as pd
import numpy as np
import re, sys, os
sys.path.append("../App")
from download_DB import download_db


In [6]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

##  Cria conexões e baixa banco de dados

In [7]:
directory = download_db(lista_download = ["skate_ug" ,"skate_usinas","skate_leilao"], force_download=False,queries_path="..//App/Queries")
skate_ug = pd.read_parquet(f"{directory}skate_ug.gzip")
skate_usinas = pd.read_parquet(f"{directory}skate_usinas.gzip")
skate_leilao = pd.read_parquet(f"{directory}skate_leilao.gzip")

Carregando arquivo de log...
skate_ug já foi baixado hoje: Dia: 20/12/22 - Horário: 13:09:42.  Portanto não foi baixado novamente.
skate_usinas já foi baixado hoje: Dia: 20/12/22 - Horário: 13:09:46.  Portanto não foi baixado novamente.
'skate_leilao.gzip' salvo em 'C:\Users\viniciusbarros\Documents\Previsor\SKATE_Downloads/2022_12_20/'.


### Importa dataframes que foram salvos

In [8]:
# Carrega dataframe com dados do bdworgs
bdworgs = pd.read_parquet("./Intermediarios/bdworgs.gzip") 

### Tratamento de dados


In [9]:
# Carrega tabela de associações
associassoes = pd.read_excel("./Inputs/associassoes.xlsx")
associassoes

,bdworgs,skate
0,Idempreendimento,IdeUsinaOutorga
1,OpTest_R_UG,DatLiberacaoSFGTeste
2,OpCom_Outor_UG,DatUGInicioOpComerOutorgado
3,OpCom_R_UG,DatLiberOpComerRealizado
4,UF,UFUsina
5,NaN,SigTipoGeracao
6,Obras_Real,DatInicioObraRealizado
7,Concretagem_Real,DatConcretagemRealizado
8,Montagem_Real,DatMontagemRealizado
9,Torres_Real,NaN


In [10]:
# Cria dicionário dos nomes das colunas para poder relacionar os nomes das colunas
# Dicionário de colunas bdworgs > skate
dict_bdworgs_skate = dict(zip(associassoes.dropna()["bdworgs"],associassoes.dropna()["skate"]))
# Dicionário de colunas skate > bdworgs
dict_skate_bdworgs = dict(zip(associassoes.dropna()["skate"],associassoes.dropna()["bdworgs"]))

In [11]:
# Colunas carregados do skate
skate_cols = [el for el in associassoes["skate"].dropna()]
# Colunas carregadas do bdworgs
bdworgs_cols =[el for el in associassoes["bdworgs"].dropna()]

# Colunas provenientes do BD skate_UGs
skate_ug_cols = []

# Colunas provenientes do BD skate_usinas
skate_usinas_col = []

for col in skate_cols:
    if col in skate_ug.columns:
        skate_ug_cols.append(col)
    if col in skate_usinas.columns:
        skate_usinas_col.append(col)
        

# Mantém apenas colunas que serão utilizadas        
bdworgs = bdworgs[bdworgs_cols]
skate_ug = skate_ug[skate_ug_cols]
skate_usinas = skate_usinas[skate_usinas_col]
skate_leilao = skate_leilao[["IdeUsinaOutorga","DatInicioSuprimento"]]


# Em algum momento as informações sobre torres passaram a ser colocadas em montagem 
mask = bdworgs.Montagem_Real.isnull() 
bdworgs.loc[mask,"Montagem_Real"] = bdworgs.loc[mask,"Torres_Real"]

# Renomeia colunas do bdworgs para o padrão do skate
bdworgs.rename(columns=dict_bdworgs_skate,inplace=True)

# A data operação comercial no bdworgs é na verdade a data de obrigação de operação comercial
# Dat_OC_obrigacao = OC_outorga (Fora do ACR) ou OC_suprimento (ACR) 
bdworgs.rename(columns={"DatUGInicioOpComerOutorgado": "Dat_OC_obrigacao"},inplace=True)

bdworgs.sample(5)

,IdeUsinaOutorga,DatLiberacaoSFGTeste,Dat_OC_obrigacao,DatLiberOpComerRealizado,UFUsina,DatInicioObraRealizado,DatConcretagemRealizado,DatMontagemRealizado,Torres_Real,DscComercializacaoEnergia,Num_UG,Pot_UG
904,32318,2017-06-03,2017-10-01,2017-11-04,BA,2016-05-15,nan,2016-09-15,nan,ACR,1 a 14,28.0
385,30926,2018-10-11,2016-01-01,2018-12-22,CE,2015-02-15,2015-05-12,nan,2015-07-15,ACR,1 a 13,27.3
1316,35715,2017-01-25,2017-01-14,2017-02-01,PA,2016-11-15,nan,2016-12-05,nan,ACR,6,1.095
72,30450,2011-04-06,nan,2011-04-09,MG,2007-07-15,nan,nan,nan,Fora do ACR,1,11.4
1346,37358,2019-09-21,2021-10-01,2019-10-25,MT,2018-05-13,2018-11-05,2019-04-02,nan,ACR,1,2.5


In [12]:
# Ordena dataframe skate_leilao por DatInicioSuprimento
skate_leilao.sort_values(by="DatInicioSuprimento",inplace=True)

# Remove linhas que possuíam datas de leilão para uma mesma usina e mantém apenas aquelas com a primeira data de 
#de DatInicioSuprimento
skate_leilao.drop_duplicates(keep="first",subset= ["IdeUsinaOutorga"],inplace=True)

# Adiciona coluna DatInicioSuprimento ao dataframe skate_usinas
skate_usinas = pd.merge(skate_usinas,
        skate_leilao[["DatInicioSuprimento","IdeUsinaOutorga"]],
        on="IdeUsinaOutorga")


skate_usinas[["DscComercializacaoEnergia","DatInicioSuprimento"]].sample(10)

,DscComercializacaoEnergia,DatInicioSuprimento
2659,Fora do ACR,NaT
2640,Fora do ACR,NaT
500,Fora do ACR,NaT
2238,Fora do ACR,NaT
1221,Fora do ACR,NaT
47,Fora do ACR,NaT
2875,Fora do ACR,NaT
2764,Fora do ACR,NaT
290,ACR,2019-01-01
301,ACR,2023-01-01


### Adiciona as seguintes  colunas no bdworgs:
- Quant_UG  &rarr; Quantidade de UGs agrupadas
- SigTipoGeracao &rarr; Tipo de geração


In [13]:
def corrige_ug(ug): # Função criada para corrigir nomes das UGs
    ug = ug.replace(" ","")
    ug = ug.replace("e",",")
    ug = ug.replace("-","a")
    ug = re.sub(r",+",r",",ug)
    ug = re.sub(r"([,ae])(0)",r"\1",ug)
    ug = re.sub(r"^0",r"",ug)
    ug = ug.split("a")
    lista = []
    for i in range(len(ug)-1):
        menor = int(ug[i].split(",")[-1])
        maior = int(ug[i+1].split(",")[0])
        list_nums = list(range(menor+1,maior))
        lista += list_nums
    ug = [el.split(",") for el in ug]
    ug = [int(el2) for el1 in ug for el2 in el1]
    ug+= lista
    return (sorted(ug))


# Insere coluna Quant_UG em bdworgs
bdworgs.insert(3,"Quant_UG",bdworgs["Num_UG"].apply(lambda x : len(corrige_ug(x))))
# Remove coluna Num_UG
bdworgs = bdworgs.drop("Num_UG",axis=1)

bdworgs.IdeUsinaOutorga = bdworgs.IdeUsinaOutorga.astype(int)
bdworgs.Quant_UG = bdworgs.Quant_UG.astype(int)
bdworgs.Pot_UG = bdworgs.Pot_UG.astype(float)

ceg_df = pd.read_excel("./Inputs/Relatorio_Outorgas.xlsx")[["Identificador da Usina","Tipo de Geração"]]
dict_id_ceg = dict(zip(ceg_df["Identificador da Usina"],ceg_df["Tipo de Geração"]))
bdworgs["SigTipoGeracao"] = bdworgs["IdeUsinaOutorga"].map(dict_id_ceg)
bdworgs.sample(5)

,IdeUsinaOutorga,DatLiberacaoSFGTeste,Dat_OC_obrigacao,Quant_UG,DatLiberOpComerRealizado,UFUsina,DatInicioObraRealizado,DatConcretagemRealizado,DatMontagemRealizado,Torres_Real,DscComercializacaoEnergia,Pot_UG,SigTipoGeracao
12,30354,2018-11-10,2017-11-30,1,2018-11-28,PA,2011-08-18,2013-03-30,2013-11-12,nan,ACR,611.11000,UHE
671,31607,2016-02-05,2018-05-01,2,2016-03-11,BA,2015-09-15,2015-10-08,nan,2015-11-01,ACR,4.70000,EOL
1708,28803,2010-08-14,2006-11-01,1,2010-12-09,GO,2007-09-01,2008-04-01,2008-08-15,nan,Fora do ACR,13.50000,PCH
374,30915,2015-11-06,2015-12-31,5,2015-12-09,RN,2014-12-01,2015-01-02,nan,2015-06-19,ACR,15.00000,EOL
1047,33193,2017-08-18,2017-07-01,11,2017-10-21,MG,2016-12-15,nan,2017-01-15,nan,ACR,10.64516,UFV


### No BD skate_ug, linhas com todas as colunas iguais tais como IdeUsinaOutorga, DatInicioOpTesteOutorgada... (exceto  MdaPotenciaUnitaria, que não necessariamente tem que ser igual em todas as linhas)  são agrupadas e são criadas duas outras colunas: 
- Quant_UG -> &rarr; Quantidade de UGs agrupadas
- Pot_UG -> &rarr; Soma da potência das UGs que foram agrupadas

#### Entrada: skate_ug
#### Saída: skate_ug_grouped

In [14]:
skate_ug_group_list = [x for x  in skate_ug.columns.to_list() if x not in ["MdaPotenciaUnitaria"]]
skate_ug_grouped = skate_ug.groupby(skate_ug_group_list,dropna=False).agg(
    Quant_UG = ("MdaPotenciaUnitaria","count"),
    Pot_UG = ("MdaPotenciaUnitaria","sum")
    
).reset_index()

### Realiza o processo acima para o bdworgs:
- Quant_UG -> &rarr; Quantidade de UGs agrupadas. Nesse caso, como o bdworgs, já possuía a coluna Quant_UG, os valores dessa coluna foram somados para os valores agrupados.
- Pot_UG -> &rarr; Soma da potência dos conjuntos de UGs que foram agrupados.

#### Entrada: bdworgs
#### Saída: bdworgs_ug_grouped

In [15]:
dummy_bdworgs = bdworgs.rename(columns={"Quant_UG":"Quant_UG_","Pot_UG":"Pot_UG_"})
dummy_bdworgs_group_list = [x for x  in dummy_bdworgs.columns.to_list() if x not in ["Quant_UG_","Pot_UG_"]]

bdworgs_ug_grouped = dummy_bdworgs.groupby(dummy_bdworgs_group_list,dropna=False).agg(
    Quant_UG = ("Quant_UG_","sum"),
    Pot_UG = ("Pot_UG_","sum")
    
).reset_index() 

### Checa se a quantidade de potência e de usinas nos dataframes iniciais e agrupados são iguais.

In [16]:
print(skate_ug_grouped.Quant_UG.sum() - skate_ug.MdaPotenciaUnitaria.count())
print(skate_ug_grouped.Pot_UG.sum() - skate_ug.MdaPotenciaUnitaria.sum())
print(bdworgs_ug_grouped.Pot_UG.sum() - bdworgs_ug_grouped.Pot_UG.sum())
print(bdworgs_ug_grouped.Quant_UG.count() - bdworgs_ug_grouped.Quant_UG.count())


0
1.4901161193847656e-08
0.0
0


### Junta bancos de dados skate_ug e skate_usina pela coluna IdeUsinaOutorga. Todos os atributos de cada usina são passados para cada UG pertecente a usina.

#### Entrada: skate_ug_grouped & skate_usinas
#### Saída: skate_merged

In [17]:
skate_merged  = pd.merge(skate_ug_grouped.drop(["SigTipoGeracao"],axis=1),
        skate_usinas,
        on="IdeUsinaOutorga",
         how="outer")
mask  = skate_merged.DscComercializacaoEnergia == "ACR"
skate_merged.loc[mask,"DatUGInicioOpComerOutorgado"] = skate_merged.DatInicioSuprimento
skate_merged.sample(10)

,IdeUsinaOutorga,DatLiberacaoSFGTeste,DatUGInicioOpComerOutorgado,DatLiberOpComerRealizado,Quant_UG,Pot_UG,UFUsina,SigTipoGeracao,DatInicioObraRealizado,DatConcretagemRealizado,DatMontagemRealizado,DscComercializacaoEnergia,DatInicioSuprimento
1346,35851,NaT,2025-04-05,NaT,22.0,22000.00,BA,UFV,NaT,NaT,NaT,Fora do ACR,NaT
2095,40195,NaT,2021-09-01,NaT,44.0,44000.00,CE,UFV,NaT,NaT,NaT,Fora do ACR,NaT
1646,37686,2019-11-18,2017-01-01,NaT,5.0,370.00,AM,UTE,2018-12-03,NaT,2019-04-24,ACR,2017-01-01
162,30054,2022-07-02,2022-05-01,2022-07-20,1.0,9000.00,ES,UTE,2021-12-09,NaT,2022-04-02,ACR,2022-05-01
1794,37949,2021-03-26,2023-01-01,2021-07-08,3.0,12600.00,RN,EOL,2020-03-23,2020-07-26,2020-11-27,ACR,2023-01-01
4071,54616,NaT,2026-01-01,NaT,33.0,49999.95,MG,UFV,NaT,NaT,NaT,Fora do ACR,NaT
998,34654,NaT,2027-02-14,NaT,12.0,50400.00,BA,EOL,NaT,NaT,NaT,Fora do ACR,NaT
3445,49594,NaT,2024-10-01,NaT,12.0,50400.00,RN,EOL,NaT,NaT,NaT,Fora do ACR,NaT
3271,49181,NaT,2023-03-20,NaT,14.0,77000.00,RN,EOL,NaT,NaT,NaT,Fora do ACR,NaT
1314,35810,2019-10-10,2017-01-01,2020-01-24,3.0,891.00,AM,UTE,2019-01-14,NaT,2019-07-01,ACR,2017-01-01


In [18]:
# Cria colunas Dat_OC_obrigacao em skate_merged
mask_ACR = skate_merged.DscComercializacaoEnergia == "ACR"
skate_merged.loc[mask_ACR,"Dat_OC_obrigacao"] = skate_merged.DatInicioSuprimento
skate_merged.loc[~mask_ACR,"Dat_OC_obrigacao"] = skate_merged.DatUGInicioOpComerOutorgado

### Converte colunas cujos nomes iniciam em "Dat" para tipo de data. É necessário que as colunas estejam nomeadas no padrão do skate.

In [19]:
colunas_datas = []
for col in set(bdworgs_ug_grouped.columns.to_list()  + skate_merged.columns.to_list()):
    if col[0:3] == "Dat":
        colunas_datas.append(col)
        print(col)
        try:
            skate_merged[col] = pd.to_datetime(skate_merged[col],format="%Y-%m-%d",errors="coerce")
        except KeyError:
            pass
        try:
            bdworgs_ug_grouped[col] = pd.to_datetime(bdworgs_ug_grouped[col],format="%Y-%m-%d",errors="coerce")
        except KeyError:
            pass

DatInicioSuprimento
DatUGInicioOpComerOutorgado
DatLiberOpComerRealizado
DatMontagemRealizado
DatConcretagemRealizado
DatInicioObraRealizado
DatLiberacaoSFGTeste
Dat_OC_obrigacao


### Checa quais colunas não estão presentes em cada um dos dataframes

In [21]:
print("Colunas que estão no skate, porém não no bdworgs:")
for col in skate_merged.columns:
    if col not in bdworgs.columns:
        print(col)
        
print("-"*10)
print("Colunas que estão no bdworgs, porém não no skate:")
for col in bdworgs.columns:
    if col not in skate_merged.columns:
        print(col)

Colunas que estão no skate, porém não no bdworgs:
DatUGInicioOpComerOutorgado
DatInicioSuprimento
----------
Colunas que estão no bdworgs, porém não no skate:
Torres_Real


### Junta BDs skate e bdworgs.
#### Entrada: skate_merged & bdworgs_ug_grouped
#### Saída: skate_merged

In [22]:
bdworgs_ug_grouped["origem"] = "bdworgs"
skate_merged["origem"] = "skate"
bd_combined = pd.concat([bdworgs_ug_grouped,skate_merged],axis=0,ignore_index=True)
#bd_combined = bdworgs_ug_grouped.copy(deep=True)
bd_combined["origem"] = bd_combined["origem"].astype("category")
bd_combined.DscComercializacaoEnergia = bd_combined.DscComercializacaoEnergia.astype("category")
bd_combined.sample(10)

,IdeUsinaOutorga,DatLiberacaoSFGTeste,Dat_OC_obrigacao,DatLiberOpComerRealizado,UFUsina,DatInicioObraRealizado,DatConcretagemRealizado,DatMontagemRealizado,Torres_Real,DscComercializacaoEnergia,SigTipoGeracao,Quant_UG,Pot_UG,origem,DatUGInicioOpComerOutorgado,DatInicioSuprimento
3381,35071,NaT,2026-07-01,NaT,AM,NaT,NaT,NaT,NaN,ACR,UTE,1.0,115205.00,skate,2026-07-01,2026-07-01
1297,30895,2015-11-06,2015-12-31,2015-12-09,RN,2014-12-01,2015-01-02,NaT,2015-06-03,ACR,EOL,3.0,9.00,bdworgs,NaT,NaT
5117,46890,NaT,2023-05-30,NaT,BA,NaT,NaT,NaT,NaN,Fora do ACR,UFV,24.0,48000.00,skate,2023-05-30,NaT
5859,49895,NaT,2024-09-18,NaT,CE,NaT,NaT,NaT,NaN,Fora do ACR,EOL,10.0,42000.00,skate,2024-09-18,NaT
6401,54646,NaT,2026-01-01,NaT,MG,NaT,NaT,NaT,NaN,Fora do ACR,UFV,33.0,49999.95,skate,2026-01-01,NaT
1134,30609,2015-12-30,2016-01-15,2016-03-02,RN,2014-07-28,2014-07-28,NaT,2015-01-15,ACR,EOL,15.0,30.00,bdworgs,NaT,NaT
5542,48918,NaT,2023-01-31,NaT,BA,NaT,NaT,NaT,NaN,Fora do ACR,UFV,11.0,37807.00,skate,2023-01-31,NaT
2309,659,NaT,2021-01-01,NaT,SC,2012-12-31,2014-01-30,2014-10-29,NaN,ACR,PCH,1.0,648.00,skate,2021-01-01,2021-01-01
6498,55592,NaT,2022-05-01,NaT,MT,2022-04-19,NaT,2022-06-11,NaN,ACR,UTE,46.0,57200.00,skate,2022-05-01,2022-05-01
1732,31764,2016-11-30,2018-01-01,2016-12-13,PE,2016-02-16,2016-04-19,NaT,2016-09-30,ACR,EOL,10.0,23.00,bdworgs,NaT,NaT


In [23]:
bd_combined[["IdeUsinaOutorga","origem","DscComercializacaoEnergia","DatUGInicioOpComerOutorgado","Dat_OC_obrigacao","DatInicioSuprimento"]]

,IdeUsinaOutorga,origem,DscComercializacaoEnergia,DatUGInicioOpComerOutorgado,Dat_OC_obrigacao,DatInicioSuprimento
0,12,bdworgs,ACR,NaT,2007-03-01,NaT
1,12,bdworgs,ACR,NaT,2008-09-01,NaT
2,19,bdworgs,Fora do ACR,NaT,2002-01-01,NaT
3,42,bdworgs,ACR,NaT,2003-11-30,NaT
4,42,bdworgs,ACR,NaT,2005-08-31,NaT
...,...,...,...,...,...,...
6599,50960,skate,Fora do ACR,NaT,NaT,NaT
6600,53724,skate,Fora do ACR,NaT,NaT,NaT
6601,53725,skate,Fora do ACR,NaT,NaT,NaT
6602,53726,skate,Fora do ACR,NaT,NaT,NaT


In [24]:
# Exporta dataframe
bd_combined.to_parquet("./Intermediarios/bd_combined.gzip")